## loading data from file

In [2]:
import numpy as np
import tensorflow as tf

In [7]:
xy = np.loadtxt('DeepLearningZeroToAll-master/data-01-test-score.csv', delimiter =',', dtype = np.float32)
# 파이썬은 slicing이 쉽다 string이든 arrray든 2차원도 가능
# : 전체를 가져와서 마지막행 제외하고
x_data = xy[:, 0:-1]
# : 전체를 가져와서 마지막행만
y_data = xy[:, [-1]]


#데이터 확인
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(25, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]
 [ 69.  74.  77.]
 [ 47.  56.  60.]
 [ 87.  79.  90.]
 [ 79.  70.  88.]
 [ 69.  70.  73.]
 [ 70.  65.  74.]
 [ 93.  95.  91.]
 [ 79.  80.  73.]
 [ 70.  73.  78.]
 [ 93.  89.  96.]
 [ 78.  75.  68.]
 [ 81.  90.  93.]
 [ 88.  92.  86.]
 [ 78.  83.  77.]
 [ 82.  86.  90.]
 [ 86.  82.  89.]
 [ 78.  83.  85.]
 [ 76.  83.  71.]
 [ 96.  93.  95.]] 25
(25, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


In [12]:
x = tf.placeholder(tf.float32, shape = [None, 3])
y = tf.placeholder(tf.float32, shape = [None, 1])

w = tf.Variable(tf.random_normal([3, 1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")

hypothesis = tf.matmul(x, w) + b

#simplified cost function
cost = tf.reduce_mean(tf.square(hypothesis - y))

#minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 1e-5)
train = optimizer.minimize(cost)

#학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dict={x: x_data, y: y_data})
    if step % 100 == 0:
        print(step, "cost : ", cost_val, "\nprediction:\n", hy_val)

print("my score will be ", sess.run(hypothesis, feed_dict={x: [[100, 70, 101]]}))
print("other scores will be ", sess.run(hypothesis, feed_dict={x: [[60, 70, 110], [90, 100, 80]]}))

0 cost :  3603.9087 
prediction:
 [[213.07008]
 [250.3493 ]
 [249.72278]
 [271.9941 ]
 [189.27649]
 [138.50937]
 [204.93452]
 [154.06064]
 [232.45819]
 [214.58224]
 [195.69873]
 [190.3082 ]
 [257.98026]
 [213.97253]
 [204.98792]
 [254.5542 ]
 [201.96136]
 [247.47449]
 [246.82547]
 [221.38095]
 [239.65308]
 [235.12677]
 [228.96469]
 [214.76035]
 [260.44934]]
100 cost :  16.807775 
prediction:
 [[156.60559 ]
 [182.7503  ]
 [182.97395 ]
 [199.36397 ]
 [137.73422 ]
 [100.63974 ]
 [150.58936 ]
 [113.793976]
 [169.42245 ]
 [156.36188 ]
 [143.35318 ]
 [138.83646 ]
 [188.95834 ]
 [156.52104 ]
 [150.43982 ]
 [185.99385 ]
 [147.25082 ]
 [182.23398 ]
 [180.97176 ]
 [162.44043 ]
 [175.94003 ]
 [171.75558 ]
 [168.17844 ]
 [157.68694 ]
 [190.31874 ]]
200 cost :  15.901815 
prediction:
 [[156.4535 ]
 [182.82277]
 [182.9088 ]
 [199.35442]
 [137.80223]
 [100.81444]
 [150.6019 ]
 [113.82324]
 [169.6244 ]
 [156.7075 ]
 [143.37828]
 [138.99736]
 [188.8327 ]
 [156.34601]
 [150.49733]
 [186.10231]
 [147.088

In [19]:
#numpy로도 큰파일올리면 메모리가 부족할 수 이씅ㅁ
# 그럴경우 텐서플로우는 queue runners를 제공
# 옵션도 여러가지 있으니 필요할 경우 사용!
filename_queue = tf.train.string_input_producer(
    ['DeepLearningZeroToAll-master/data-01-test-score.csv'], shuffle = False, name = "filename_queue")

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)


train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size = 10)

x = tf.placeholder(tf.float32, shape = [None, 3])
y = tf.placeholder(tf.float32, shape = [None, 1])

w = tf.Variable(tf.random_normal([3, 1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")

hypothesis = tf.matmul(x, w) + b

cost = tf.reduce_mean(tf.square(hypothesis - y))

#minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess= sess, coord = coord)
sess.run(tf.global_variables_initializer())
for step in range(2001):
    x_batch , y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train],
                                  feed_dict={x: x_batch, y: y_batch})
    if step % 100 == 0:
        print(step, "cost : ", cost_val, "\nprediction:\n", hy_val)
        
coord.request_stop()
coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
0 cost :  1029.8069 
prediction:
 [[181.42961]
 [225.86765]
 [218.6053 ]
 [236.55   ]
 [175.56946]
 [128.2059 ]
 [172.76483]
 [121.78403]
 [211.14314]
 [193.0335 ]]
100 cost :  35.55661 
prediction:
 [[151.24213]
 [189.47324]
 [182.79562]
 [197.6394 ]
 [147.71172]
 [107.79162]
 [143.80246]
 [100.54511]
 [177.21262]
 [161.79204]]
200 cost :  33.957302 
prediction:
 [[151.25685]
 [189.3843 ]
 [182.75581]
 [197.68288]
 [147.5457 ]
 [107.79082]
 [143.99417]
 [100.91289]
 [177.21089]
 [161.97292]]
300 cost :  32.443794 
prediction:
 [[151.27304]
 [189.29655]
 [182.71773]
 [197.72514]
 [147.38306]
 [107.78768]
 [144.18054]
 [101.27053]
 [177.2064 ]
 [162.1444 ]]
400 cost :  31.011078 
prediction:
 [[151.29057]
 [189.21007]
 [182.6813 ]
 [197.76624]
 [147.2237 ]
 [107.78234]
 [144.36174]
 [101.61832]
 [177.19939]
 [162.30692]]
500 cost :  29.654413 
prediction:
 [[151.30933 ]
 [189.12482 ]
 [182.64645 ]
 [197.8